In [1]:
import time

import numpy as np
import pandas as pd
import selenium
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import (NoSuchElementException,
                                        StaleElementReferenceException,
                                        TimeoutException)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.linkedin.com')

In [4]:
with open('./login_credential.txt') as f:
    lines = f.read().splitlines()

In [5]:
username = driver.find_element(By.ID, 'session_key')
password = driver.find_element(By.ID, 'session_password')

In [6]:
username.send_keys(lines[0])

time.sleep(5)
password.send_keys(lines[1])

In [7]:
driver.find_element(By.CLASS_NAME, 'sign-in-form__submit-button').click()

In [8]:
driver.get('https://linkedin.com/jobs')
time.sleep(10)

In [9]:
job_search = driver.find_element(By.XPATH, '//*[@id="jobs-search-box-keyword-id-ember25"]')
job_search.send_keys('data')

time.sleep(2)

In [10]:
location_search = driver.find_element(By.XPATH, '//*[@id="jobs-search-box-location-id-ember25"]')
location_search.send_keys('United Kingdom')

time.sleep(2)

In [11]:
job_search.send_keys(Keys.RETURN)
time.sleep(5)

In [12]:
def get_job_elems(driver):
    return driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

job_list = driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

In [13]:
driver.refresh()

In [14]:

job_list = driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

In [15]:
len(job_list)

0

In [16]:
import re

def get_id(driver):
    link_elem = driver.find_element(By.XPATH, 
                        '//div[contains(@class, "jobs-unified-top-card t-14")]//a[@href]')
    link = link_elem.get_attribute('href')
    _id = re.findall('(?<=view/)(.*)(?=/)', link)

    assert len(_id) == 1, "Multiple Id found for same job posting!"

    return int(_id[0]) 

In [38]:
def get_skills(driver):
    time.sleep(0.5)
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, 
                        "//button[contains(@aria-label, 'View strong skill match modal')]"))
        )
        try:
            driver.find_element(By.XPATH, 
                            "//button[contains(@aria-label, 'View strong skill match modal')]").click()
        except StaleElementReferenceException:
            driver.find_element(By.XPATH, 
                            "//button[contains(@aria-label, 'View strong skill match modal')]").click()
    except TimeoutException:
        return False

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, 
                        "//ul[contains(@class, 'job-details-skill-match-status-list')]"))
        )
    except TimeoutException:
        return

    skill = driver.find_elements(By.XPATH, 
                            "//ul[contains(@class, 'job-details-skill-match-status-list')]")

    text = skill[0].text.replace("Add", '')

    driver.find_element(By.XPATH, "//div[contains(@aria-labelledby, 'jobs-skill-match-modal-header')]//button").click()

    return [skill for skill in text.split("\n") if skill != '']
    
    

In [18]:
def generate_skill_df(elem):
    action = ActionChains(driver)
    action.move_to_element(elem).perform()
    elem.click()
    _id = get_id(elem)
    _skills = get_skills(driver)

    if _skills == False:
        return 

    _df = pd.DataFrame({'id': [_id for x in range(len(_skills))], \
                        'skills': _skills})

    return(_df)

In [19]:
def generate_all_skill_df(driver, elems):
    df_list = []
    for elem in elems:
        _df = generate_skill_df(elem)
        df_list.append(_df)
    
    df_list = [df for df in df_list if ~isinstance(df, type(None))]
    return pd.concat(df_list)

In [20]:
job_list = get_job_elems(driver)
# test_all = generate_all_skill_df(driver, job_list)

In [21]:
get_skills(driver)

False

In [22]:
soup = bs(driver.page_source)
test = soup.find_all(class_=re.compile('jobs-unified-top-card__job-insight'))
tuple([y for y in [x.find(string=re.compile('.*employees.*')) for x in test] if y][0].split('·') )

('11-50 employees',)

In [23]:
def scrape_job(driver):
    soup = bs(driver.page_source)

In [24]:
def set_execute(func):
    func.execute = True
    return func

class linkedin_soup(object):

    def __init__(self, soup, auto_execute = True):
        self.soup = soup
        if auto_execute:
            self.auto_generate()
    
    @set_execute
    def get_link(self):
        self.link = 'https://www.linkedin.com' + self.soup.find(class_=re.compile('jobs-unified-top-card__content.*')).\
                                                        find('a')['href']

    @set_execute
    def get_id(self):
        if 'link' not in self.__dict__.keys():
            self.get_link()
        _id = re.findall('(?<=view/)(.*)(?=/)', self.link)

        assert len(_id) == 1, 'Multiple id detected'
        self.id = int(_id[0])


    @set_execute
    def get_title(self):
        title_elem = self.soup.find_all(class_='t-24 t-bold jobs-unified-top-card__job-title')
        assert len(title_elem) == 1, "More than 1 job title found on page!"

        title = title_elem[0].contents[0]
        assert isinstance(title, str)

        self.title = title

    @set_execute
    def get_company(self):
        try:
            self.company =  self.soup.find(class_ = 'jobs-unified-top-card__company-name').text.strip()
        except AttributeError:
            self.company_name =  np.nan

    @set_execute
    def get_location(self):
        try:
            self.location =  self.soup.find(class_="jobs-unified-top-card__bullet").text.strip()
        except AttributeError:
            self.location =  np.nan

    @set_execute
    def get_job_type(self):
        try:
            self.job_type =  self.soup.find(class_=re.compile(".*workplace-type")).text
        except AttributeError:
            self.job_type =  np.nan
        
    @set_execute
    def get_employees_company(self):
        insight = self.soup.find_all(class_=re.compile('jobs-unified-top-card__job-insight'))
        emp_comp = [string for string in [match.find(string=re.compile(".*employees.*")) 
                                for match in insight] if string][0]
        try:
            self.employee, self.company_type = tuple([data.strip() for data in emp_comp.split("·")])
        except ValueError:
            self.employee = emp_comp.strip()
            self.company_type = np.nan

    @set_execute
    def get_salary(self):
        try:
            self.salary = self.soup.find(href='#SALARY').text
        except AttributeError:
            self.salary = np.nan

    @set_execute
    def get_job_desc(self):
        self.job_desc = self.soup.select("div#job-details > span")[0].get_text(separator="\n").strip()
    
    def auto_generate(self):
        for method in dir(self):
            if getattr(getattr(self, method), 'execute', False):
                getattr(self, method)()
    
    def generate_df(self):
        return pd.DataFrame({key:item for key, item in self.__dict__.items() if key != 'soup'}, 
            index = [0])

        


In [25]:
soup = bs(driver.page_source)
test = linkedin_soup(soup)

In [26]:
test.generate_df()

,company,employee,company_type,link,id,job_desc,job_type,location,salary,title
0,Camden A.I.,11-50 employees,NaN,https://www.linkedin.com/jobs/view/3483453445/...,3483453445,We are looking for a dynamic energetic person ...,Hybrid,"London, England, United Kingdom",NaN,Business Intelligence Lead


In [27]:
pd.DataFrame({key:item for key, item in test.__dict__.items() if (key != 'soup') & (key != 'id')}, 
            index = [test.__dict__['id']])

,company,employee,company_type,link,job_desc,job_type,location,salary,title
3483453445,Camden A.I.,11-50 employees,NaN,https://www.linkedin.com/jobs/view/3483453445/...,We are looking for a dynamic energetic person ...,Hybrid,"London, England, United Kingdom",NaN,Business Intelligence Lead


In [28]:
test_link = soup.find(class_=re.compile('jobs-unified-top-card__content.*')).find('a')['href']


In [29]:
import webbrowser

webbrowser.open('https://www.linkedin.com' + test_link)

True

In [30]:
test.auto_generate()

In [31]:
test.get_company()
test.get_location()

In [32]:
def wrapper(func):
    func.is_test = True
    return func

@wrapper
def blah(x):
    return

blah(1)

blah.what_the = True


In [33]:
class listing_wait(object):

    def __init__(self, locator, xpath):
        self.locator = locator
        self.xpath = xpath

    def __call__(self, driver):
        _elements = driver.find_elements(self.locator, self.xpath)
        if len(_elements) == 0:
            return False
        else:
            return _elements

In [34]:

# driver.refresh()
# # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 
# #                         "//ul[contains(@class, 'scaffold-layout__list-container')]")))

# time.sleep(15)

# job_elems = get_job_elems(driver)
# print(len(job_elems))
# action = ActionChains(driver)

# for elem in job_elems:
#     action.move_to_element(elem).perform()
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable(elem))
#     elem.click()
#     # test = WebDriverWait(driver, 10).until(listing_wait(By.XPATH, 
#     #                     "//h2[contains(@class, 't-24 t-bold jobs-unified-top-card__job-title')]"))
#     WebDriverWait(driver, 10, ignored_exceptions=StaleElementReferenceException).until(EC.element_to_be_clickable((By.XPATH, 
#                                 "//div[contains(@class, 'jobs-apply-button')]//button")))
#     # time.sleep(1)
#     soup = bs(driver.page_source)
#     print(get_title(soup), get_company(soup), get_location(soup))

soup = bs(driver.page_source)

In [47]:
def scrape_page(driver):
    driver.refresh()
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 
    #                         "//ul[contains(@class, 'scaffold-layout__list-container')]")))

    time.sleep(20)

    job_elems = get_job_elems(driver)
    assert len(job_elems) == 25, "Job listing less than 25, web page not properly loaded!"

    skill_dfs = []
    job_dfs = []
    for elem in job_elems:
        action = ActionChains(driver)
        action.move_to_element(elem).perform()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(elem))
        elem.click()
        # test = WebDriverWait(driver, 10).until(listing_wait(By.XPATH, 
        #                     "//h2[contains(@class, 't-24 t-bold jobs-unified-top-card__job-title')]"))
        try:
            WebDriverWait(driver, 20, ignored_exceptions=StaleElementReferenceException).until(EC.element_to_be_clickable((By.XPATH, 
                                        "//div[contains(@class, 'jobs-apply-button')]//button")))
        except TimeoutException:
            WebDriverWait(driver, 20, ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.XPATH, 
                                        "//div[contains(@class, 'feedback--success')]")))

        time.sleep(0.5)
        
        skill_dfs.append(generate_skill_df(elem))
        _soup = bs(driver.page_source)
        _linkedin_soup = linkedin_soup(_soup)
        job_dfs.append(_linkedin_soup.generate_df())

    return (pd.concat(skill_dfs, ignore_index=True), pd.concat(job_dfs, ignore_index=True))



In [54]:
def scrape_pages(driver, page_num):
    skill_dfs = []
    job_dfs = []

    try:
        for page in range(1, page_num + 1):
            print(f"Page {page}")
            page_button = driver.find_element(By.XPATH, f"//button[contains(@aria-label, 'Page {page}')]")
            action = ActionChains(driver)
            action.move_to_element(page_button)
            page_button.click()

            time.sleep(15)

            _skill_df, _job_df = scrape_page(driver)

            skill_dfs.append(_skill_df)
            job_dfs.append(_job_df)
    except Exception as err:
       print(f"Unexpected {err=}, {type(err)=}")
    finally:
        return (pd.concat(skill_dfs, ignore_index=True), pd.concat(job_dfs, ignore_index=True))

In [55]:
skill_df, job_df = scrape_pages(driver, 40)

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Unexpected err=IndexError('list index out of range'), type(err)=<class 'IndexError'>


In [57]:
job_df

,company,employee,company_type,link,id,job_desc,job_type,location,salary,title
0,Haley Bridge,1-10 employees,Staffing and Recruiting,https://www.linkedin.com/jobs/view/3484754630/...,3484754630,"Data Engineer, Snowflake, ETL, Data Visualisat...",Hybrid,"London, England, United Kingdom",£80/hr - £95/hr,Data Engineer
1,Camden A.I.,11-50 employees,NaN,https://www.linkedin.com/jobs/view/3483453445/...,3483453445,We are looking for a dynamic energetic person ...,Hybrid,"London, England, United Kingdom",NaN,Business Intelligence Lead
2,Ricardo plc,"1,001-5,000 employees",Environmental Services,https://www.linkedin.com/jobs/view/3472825102/...,3472825102,Role:\n \nCustomer Data Consultant\n \nLocatio...,Hybrid,"England, United Kingdom",NaN,Customer Data Consultant
3,PHASTAR,201-500 employees,Research Services,https://www.linkedin.com/jobs/view/3475077576/...,3475077576,ABOUT PHASTAR\n \nPHASTAR is a multiple award-...,Hybrid,"England, United Kingdom",NaN,Graduate Statistician
4,Canxchange,11-50 employees,NaN,https://www.linkedin.com/jobs/view/3484746026/...,3484746026,Title: Cannabis Market Data Analyst\nEmploymen...,On-site,"London, England, United Kingdom",NaN,Data Analyst
...,...,...,...,...,...,...,...,...,...,...
220,developrec,11-50 employees,Staffing and Recruiting,https://www.linkedin.com/jobs/view/3468029153/...,3468029153,Job Title: Lead Data Analyst \nLocation: Remot...,Hybrid,"London, England, United Kingdom",NaN,Data Analyst
221,iCrossing UK,"501-1,000 employees",Advertising Services,https://www.linkedin.com/jobs/view/3476852903/...,3476852903,iCrossing combines the capability of a digital...,Hybrid,"London, England, United Kingdom",NaN,Google Analytics Specialist
222,Burberry,"10,001+ employees",Luxury Goods & Jewelry,https://www.linkedin.com/jobs/view/3484738799/...,3484738799,"Job description:\n\n At Burberry, we believe c...",Hybrid,"Leeds, England, United Kingdom",NaN,HR Data Analyst
223,Osborne Clarke,"1,001-5,000 employees",Law Practice,https://www.linkedin.com/jobs/view/3464578073/...,3464578073,Description\n \nThis is a fantastic opportunit...,NaN,"Bristol, England, United Kingdom",NaN,Data Science Apprentice 2023 - Bristol
